In [1]:
from google.colab import drive
drive.mount("/content/drive")

import os
os.chdir("/content/drive/MyDrive/PROJECT_3")

Mounted at /content/drive


In [2]:
import pandas as pd
import json

In [3]:
feature_train = pd.read_csv("train_dataset/train.feat.tsv", sep="\t", header=None, converters={1: json.loads})

In [4]:
feature_train.head()

,0,1
0,VizWiz_train_00006882,"{'objects': [{'rect': [14.31872272491455, 508...."
1,VizWiz_train_00006921,"{'objects': [{'rect': [57.81739044189453, 642...."
2,VizWiz_train_00006981,"{'objects': [{'rect': [235.05580139160156, 925..."
3,VizWiz_train_00007006,"{'objects': [{'rect': [175.97381591796875, 439..."
4,VizWiz_train_00006958,"{'objects': [{'rect': [1183.648681640625, 652...."


In [5]:
feature_train["test"] = feature_train[1].apply(lambda x: x["relations"])

In [6]:
feature_train

,0,1,test
0,VizWiz_train_00006882,"{'objects': [{'rect': [14.31872272491455, 508....",None
1,VizWiz_train_00006921,"{'objects': [{'rect': [57.81739044189453, 642....",None
2,VizWiz_train_00006981,"{'objects': [{'rect': [235.05580139160156, 925...",None
3,VizWiz_train_00007006,"{'objects': [{'rect': [175.97381591796875, 439...",None
4,VizWiz_train_00006958,"{'objects': [{'rect': [1183.648681640625, 652....",None
...,...,...,...
7995,VizWiz_train_00001050,"{'objects': [{'rect': [0.0, 0.0, 966.386657714...",None
7996,VizWiz_train_00000937,"{'objects': [{'rect': [720.6830444335938, 255....",None
7997,VizWiz_train_00001015,"{'objects': [{'rect': [194.37112426757812, 118...",None
7998,VizWiz_train_00001097,"{'objects': [{'rect': [0.0, 551.6467895507812,...",None


In [8]:
import ast

In [9]:
hw_df = pd.read_csv("train_dataset/train.hw.tsv", sep="\t", header=None, converters={1:ast.literal_eval}, index_col=0)

In [10]:
hw_df.head()

,1
0,
VizWiz_train_00006882,"[{'height': 1632, 'width': 1224}]"
VizWiz_train_00006921,"[{'height': 1632, 'width': 1224}]"
VizWiz_train_00006981,"[{'height': 1296, 'width': 968}]"
VizWiz_train_00007006,"[{'height': 1296, 'width': 968}]"
VizWiz_train_00006958,"[{'height': 2592, 'width': 1936}]"


In [11]:
import numpy as np

In [12]:
import base64

def generate_additional_features(rect, h, w):
  mask = np.array([w, h, w, h], dtype=np.float32)
  rect = np.clip(rect/mask, 0, 1)
  res = np.hstack((rect, [rect[3] - rect[1], rect[2] - rect[0]]))
  return res.astype(np.float32)

def generate_features(x):
  idx, data, num_boxes = x[0], x[1]["objects"], len(x[1]["objects"])
  h, w, features_arr = hw_df.loc[idx, 1][0]["height"], hw_df.loc[idx, 1][0]["width"], []
  
  for i in range(num_boxes):
    features = np.frombuffer(base64.b64decode(data[i]["feature"]), np.float32)
    pos_feat = generate_additional_features(data[i]['rect'], h, w)
    x = np.hstack((features, pos_feat))
    features_arr.append(x.astype(np.float32))
  
  features = np.vstack(tuple(features_arr))
  features = base64.b64encode(features).decode("utf-8")
  return {"features": features, "num_boxes": num_boxes}

def generate_labels(x):
  data = x[1]["objects"]
  res = [{"class": el["class"].capitalize(), "conf": el["conf"], "rect": el['rect']} for el in data]
  return res

In [13]:
feature_train["feature"] = feature_train.apply(generate_features, axis=1)
feature_train["feature"] = feature_train["feature"].apply(json.dumps)

feature_train["label"] = feature_train.apply(generate_labels, axis=1)
feature_train["label"] = feature_train["label"].apply(json.dumps)

In [14]:
feature_train.head()

,0,1,test,feature,label
0,VizWiz_train_00006882,"{'objects': [{'rect': [14.31872272491455, 508....",None,"{""features"": ""+6iEPn2i5z8AAAAAGTu9QFWtgj6IRjs6...","[{""class"": ""Box"", ""conf"": 0.8936614394187927, ..."
1,VizWiz_train_00006921,"{'objects': [{'rect': [57.81739044189453, 642....",None,"{""features"": ""l3KhPe8aXj8AAAAAksgzQYh0G0ANl5FA...","[{""class"": ""Drink"", ""conf"": 0.7944185137748718..."
2,VizWiz_train_00006981,"{'objects': [{'rect': [235.05580139160156, 925...",None,"{""features"": ""AAAAAAAAAAAAAAAAHwN+P5uKxT4IJVI/...","[{""class"": ""Flower"", ""conf"": 0.725602686405181..."
3,VizWiz_train_00007006,"{'objects': [{'rect': [175.97381591796875, 439...",None,"{""features"": ""Yh1KPkgdQj8AAAAAL2xZQatIlz7B1B0/...","[{""class"": ""Laptop"", ""conf"": 0.905105471611023..."
4,VizWiz_train_00006958,"{'objects': [{'rect': [1183.648681640625, 652....",None,"{""features"": ""EVOGPvNH6z8AAAAA34KFQMFpOz0CeUk+...","[{""class"": ""Wall"", ""conf"": 0.45626822113990784..."


In [15]:
os.chdir("scene_graph_benchmark")

In [16]:
from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

In [17]:
tsv_writer(feature_train[[0, "label"]].values.tolist(), "/content/drive/MyDrive/PROJECT_3/train_dataset/train.label.tsv")

tsv_writer(feature_train[[0, 'feature']].values.tolist(), "/content/drive/MyDrive/PROJECT_3/train_dataset/train.feature.tsv")

# Transform test dataset

In [18]:
feature_test = pd.read_csv("/content/drive/MyDrive/PROJECT_3/test_dataset/test.feat.tsv", sep="\t", header=None, converters={1:json.loads})

In [19]:
hw_df = pd.read_csv("/content/drive/MyDrive/PROJECT_3/test_dataset/test.hw.tsv", sep="\t", header=None, converters={1:ast.literal_eval}, index_col=0)

In [21]:
feature_test["feature"] = feature_test.apply(generate_features, axis=1)
feature_test["feature"] = feature_test["feature"].apply(json.dumps)

feature_test["label"] = feature_test.apply(generate_labels, axis=1)
feature_test["label"] = feature_test["label"].apply(json.dumps)

In [22]:
feature_test.head()

,0,1,features,feature,label
0,VizWiz_train_00009470,"{'objects': [{'rect': [500.110107421875, 940.9...",{'features': 'AAAAAAAAAAAAAAAAeAAgP3AfsUCoCwtB...,"{""features"": ""AAAAAAAAAAAAAAAAeAAgP3AfsUCoCwtB...","[{""class"": ""Letter"", ""conf"": 0.730983376502990..."
1,VizWiz_train_00008457,"{'objects': [{'rect': [95.5503158569336, 1535....",{'features': 'KyXxPgAAAAAAAAAAsIacPwAAAACdYiQ9...,"{""features"": ""KyXxPgAAAAAAAAAAsIacPwAAAACdYiQ9...","[{""class"": ""Table"", ""conf"": 0.9410740733146667..."
2,VizWiz_train_00008945,"{'objects': [{'rect': [315.16998291015625, 719...",{'features': 'AAAAAAAAAAAAAAAANBsCQI+Elj4AAAAA...,"{""features"": ""AAAAAAAAAAAAAAAANBsCQI+Elj4AAAAA...","[{""class"": ""Label"", ""conf"": 0.8551745414733887..."
3,VizWiz_train_00009814,"{'objects': [{'rect': [221.41725158691406, 718...",{'features': 'AAAAAAAAAAAAAAAAAAAAALsxwD0AAAAA...,"{""features"": ""AAAAAAAAAAAAAAAAAAAAALsxwD0AAAAA...","[{""class"": ""Carrot"", ""conf"": 0.862469077110290..."
4,VizWiz_train_00009247,"{'objects': [{'rect': [0.0, 69.5399169921875, ...",{'features': 'C5TwPgAAAAAAAAAANwwhQNFaRD8p+e0/...,"{""features"": ""C5TwPgAAAAAAAAAANwwhQNFaRD8p+e0/...","[{""class"": ""Curtain"", ""conf"": 0.62673830986022..."


In [23]:
tsv_writer(feature_test[[0, "label"]].values.tolist(), "/content/drive/MyDrive/PROJECT_3/test_dataset/test.label.tsv")

tsv_writer(feature_test[[0, 'feature']].values.tolist(), "/content/drive/MyDrive/PROJECT_3/test_dataset/test.feature.tsv")